In [ ]:
import pandas as pd

# 파일 경로
file_path = r"your_path_to_recipe.csv"

# Step 1: 기존 CSV 파일 읽기
df = pd.read_csv(file_path)

# Step 2: 불필요한 열 삭제 (link, source)
df = df.drop(columns=["link", "source"])

# 기존 파일 덮어쓰기
df.to_csv(file_path, index=False)
print(f"Updated file saved at: {file_path}")

# Step 3: 필요한 열 (title, NER)만 추출
new_df = df[["title", "NER"]]

# 새로운 파일 경로
new_file_path = r"new_recipe_path"

# 새로운 CSV 파일 저장
new_df.to_csv(new_file_path, index=False)
print(f"New file with 'title' and 'NER' saved at: {new_file_path}")


## Step 1:  NER과 95개의 클래스 간 연관성 확인

In [ ]:
import pandas as pd

# CSV 파일 경로
file_path = r"C:\Users\gusdk\Desktop\포토폴리오\YAI 학회\YAICON\24_2\dataset\title_and_ner.csv"

# 95개의 클래스 정의
classes = [
"Romaine_lettuce", "Spinach","Wakame","Chili_powder","Yellow_bell_pepper","Cooked_rice","Paprika","Vienna_sausage",
"Sausage","Tofu","Salsa","Noodle","Ramen","Soybean_sprouts","Jalapeño","Milk","Soy_sauce","Shiitake_mushrooms",
"Basil","Sweet_potato","Squid","Shrimp","Rice","Mung_bean_sprouts","Pork_belly","Lettuce","Salmon_sashimi",
"Ham","Mushroom","Lemon","Cucumber","Zucchini","Slice_cheese","Gochujang","Parsley","Grilled_beef",
"Chili_sauce","Avocado","Bread","Chicken","Baguette","Bell_pepper","Whipping_cream","Sundae_(Korean_sausage)",
"Chicken_breast","Pepper","Green_bell_pepper","Enoki_mushrooms","Banana","Almond","Tomato_pasta_sauce",
"Sugar","Canned_tuna","Chili_pepper","Somen","Carrot","Green_chili","Tomato","Celery","Radish_kimchi",
"Sesame_seeds","Napa_cabbage","Honey","Mayonnaise","Egg","Kelp","Ketchup","Jujube","Salt","Cabbage_kimchi",
"Garlic","Oyster_mushrooms","Cabbage","Crab_sticks","Hot_sauce","Spaghetti","Broccoli","Cherry_tomatoes",
"Fish","Potato","Cheese","Bacon","Red_cabbage","rice","Green_onion","Seasoned_seaweed","Steak",
"Seaweed","Green_olives","Onion","Mackerel","Eggplant","Udon","Butter","Red_bell_pepper"
]

# 데이터 로드
df = pd.read_csv(file_path)

# 재료와 클래스 매칭
relationships = []
for index, row in df.iterrows():
    title = row['title']
    ner_list = eval(row['NER'])  # 문자열을 리스트로 변환
    for ingredient in ner_list:
        if ingredient in classes:
            relationships.append((title, ingredient, "BELONGS_TO_CLASS"))

            #뒤에서 다시 불러올 예정


# 관계 데이터를 저장할 경로
output_file_path = r"relationships.csv"

# DataFrame 생성
relationships_df = pd.DataFrame(relationships, columns=["title", "ingredient", "relationship_type"])

# CSV 파일로 저장
relationships_df.to_csv(output_file_path, index=False)
print(f"Relationships saved to {output_file_path}")


## Step 2: Neo4j에 데이터 추가 및 관계 생성

In [1]:
!pip install neo4j


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

5.26.0


In [17]:
from neo4j import GraphDatabase

# driver = GraphDatabase.driver(uri, auth=(user, password))
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'yai1234!'))
session = driver.session()

In [18]:
q = 'MATCH (n) RETURN n LIMIT 5'
nodes = session.run(q)

for node in nodes:
    print(node)

session.close()

<Record n=<Node element_id='4:babad431-0528-4012-bd43-2fb32b939f5d:0' labels=frozenset({'Ingredient'}) properties={'name': 'Romaine_lettuce'}>>
<Record n=<Node element_id='4:babad431-0528-4012-bd43-2fb32b939f5d:1' labels=frozenset({'Ingredient'}) properties={'name': 'Spinach'}>>
<Record n=<Node element_id='4:babad431-0528-4012-bd43-2fb32b939f5d:2' labels=frozenset({'Ingredient'}) properties={'name': 'Wakame'}>>
<Record n=<Node element_id='4:babad431-0528-4012-bd43-2fb32b939f5d:3' labels=frozenset({'Ingredient'}) properties={'name': 'Chili_powder'}>>
<Record n=<Node element_id='4:babad431-0528-4012-bd43-2fb32b939f5d:4' labels=frozenset({'Ingredient'}) properties={'name': 'Yellow_bell_pepper'}>>


In [24]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"  # 확인된 포트 번호로 수정
user = "neo4j"
password = "yai1234!"  # 정확한 비밀번호 입력

try:
    driver = GraphDatabase.driver(uri, auth=(user, password))
    with driver.session() as session:
        result = session.run("RETURN 1 AS result")
        for record in result:
            print("Connected successfully! Result:", record["result"])
    driver.close()
except Exception as e:
    print("Failed to connect to Neo4j:", e)


Connected successfully! Result: 1


In [ ]:
import pandas as pd
import logging
from neo4j import GraphDatabase

# Logging 설정
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("neo4j_operations.log"),  # 로그를 파일에 저장
        logging.StreamHandler()                      # 터미널 출력
    ]
)

# Neo4j 연결 클래스
class IngredientGraph:
    def __init__(self, uri, user, password):
        try:
            self.driver = GraphDatabase.driver(uri, auth=(user, password))
            logging.info("Successfully connected to Neo4j!")
        except Exception as e:
            logging.error("Failed to connect to Neo4j: %s", e)
            raise

    def close(self):
        self.driver.close()
        logging.info("Neo4j connection closed.")

    def create_ingredient_node(self, ingredient):
        query = "MERGE (i:Ingredient {name: $ingredient})"
        try:
            with self.driver.session() as session:
                session.run(query, ingredient=ingredient)
            logging.info("Created or merged ingredient node: %s", ingredient)
        except Exception as e:
            logging.error("Failed to create ingredient node (%s): %s", ingredient, e)

    def create_class_node(self, class_name):
        query = "MERGE (c:Class {name: $class_name})"
        try:
            with self.driver.session() as session:
                session.run(query, class_name=class_name)
            logging.info("Created or merged class node: %s", class_name)
        except Exception as e:
            logging.error("Failed to create class node (%s): %s", class_name, e)

    def create_relationship(self, ingredient, class_name, relationship_type):
        query = f"""
        MATCH (i:Ingredient {{name: $ingredient}})
        MATCH (c:Class {{name: $class_name}})
        MERGE (i)-[:{relationship_type}]->(c)
        """
        try:
            with self.driver.session() as session:
                session.run(query, ingredient=ingredient, class_name=class_name)
            logging.info("Created relationship: (%s)-[:%s]->(%s)", ingredient, relationship_type, class_name)
        except Exception as e:
            logging.error("Failed to create relationship (%s)-[:%s]->(%s): %s", ingredient, relationship_type, class_name, e)

# CSV 파일 경로
file_path = r"new_recipe_path_title_and_ner.csv"


# 95개의 클래스 정의
classes = [
"Romaine_lettuce", "Spinach","Wakame","Chili_powder","Yellow_bell_pepper","Cooked_rice","Paprika","Vienna_sausage",
"Sausage","Tofu","Salsa","Noodle","Ramen","Soybean_sprouts","Jalapeño","Milk","Soy_sauce","Shiitake_mushrooms",
"Basil","Sweet_potato","Squid","Shrimp","Rice","Mung_bean_sprouts","Pork_belly","Lettuce","Salmon_sashimi",
"Ham","Mushroom","Lemon","Cucumber","Zucchini","Slice_cheese","Gochujang","Parsley","Grilled_beef",
"Chili_sauce","Avocado","Bread","Chicken","Baguette","Bell_pepper","Whipping_cream","Sundae_(Korean_sausage)",
"Chicken_breast","Pepper","Green_bell_pepper","Enoki_mushrooms","Banana","Almond","Tomato_pasta_sauce",
"Sugar","Canned_tuna","Chili_pepper","Somen","Carrot","Green_chili","Tomato","Celery","Radish_kimchi",
"Sesame_seeds","Napa_cabbage","Honey","Mayonnaise","Egg","Kelp","Ketchup","Jujube","Salt","Cabbage_kimchi",
"Garlic","Oyster_mushrooms","Cabbage","Crab_sticks","Hot_sauce","Spaghetti","Broccoli","Cherry_tomatoes",
"Fish","Potato","Cheese","Bacon","Red_cabbage","rice","Green_onion","Seasoned_seaweed","Steak",
"Seaweed","Green_olives","Onion","Mackerel","Eggplant","Udon","Butter","Red_bell_pepper"
]

# 데이터 로드
try:
    df = pd.read_csv(file_path)
    logging.info("CSV file loaded successfully: %s", file_path)
except Exception as e:
    logging.error("Failed to load CSV file: %s", e)
    raise

# 재료와 클래스 매칭
relationships = []
try:
    for index, row in df.iterrows():
        title = row['title']
        ner_list = eval(row['NER'])  # 문자열을 리스트로 변환
        for ingredient in ner_list:
            if ingredient in classes:
                relationships.append((title, ingredient, "BELONGS_TO_CLASS"))
    logging.info("Relationships extracted successfully. Total relationships: %d", len(relationships))
except Exception as e:
    logging.error("Failed to process relationships: %s", e)
    raise

# Neo4j 연결 설정
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "yai1234!"

graph = IngredientGraph(uri, user, password)

# 관계 생성
try:
    for title, ingredient, relationship_type in relationships:
        graph.create_ingredient_node(ingredient)
        graph.create_class_node(title)
        graph.create_relationship(ingredient, title, relationship_type)
    logging.info("All relationships created successfully.")
except Exception as e:
    logging.error("Failed to create relationships: %s", e)
finally:
    graph.close()

2024-11-17 13:03:21,179 - INFO - CSV file loaded successfully: C:\Users\gusdk\Desktop\포토폴리오\YAI 학회\YAICON\24_2\dataset\title_and_ner.csv
2024-11-17 13:06:16,876 - INFO - Relationships extracted successfully. Total relationships: 470623
2024-11-17 13:06:16,981 - INFO - Successfully connected to Neo4j!
2024-11-17 13:06:23,578 - INFO - Created or merged ingredient node: Tomato
2024-11-17 13:06:26,334 - INFO - Created or merged class node: Taco-Filled Green Pepper
2024-11-17 13:06:28,790 - INFO - Created relationship: (Tomato)-[:BELONGS_TO_CLASS]->(Taco-Filled Green Pepper)
2024-11-17 13:06:30,848 - INFO - Created or merged ingredient node: rice
2024-11-17 13:06:32,880 - INFO - Created or merged class node: Chicken Casserole
2024-11-17 13:06:34,923 - INFO - Created relationship: (rice)-[:BELONGS_TO_CLASS]->(Chicken Casserole)
2024-11-17 13:06:37,000 - INFO - Created or merged ingredient node: rice
2024-11-17 13:06:39,045 - INFO - Created or merged class node: Consomme Chicken
2024-11-17 13

KeyboardInterrupt: 

In [1]:
# 재료 목록을 딕셔너리로 정의
ingredients = {
    "Romaine_lettuce": 0,
    "Spinach": 1,
    "Wakame": 2,
    "Chili_powder": 3,
    "Yellow_bell_pepper": 4,
    "Cooked_rice": 5,
    "Paprika": 6,
    "Vienna_sausage": 7,
    "Sausage": 8,
    "Tofu": 9,
    "Salsa": 10,
    "Noodle": 11,
    "Ramen": 12,
    "Soybean_sprouts": 13,
    "Jalapeño": 14,
    "Milk": 15,
    "Soy_sauce": 16,
    "Shiitake_mushrooms": 17,
    "Basil": 18,
    "Sweet_potato": 19,
    "Squid": 20,
    "Shrimp": 21,
    "Rice": 22,
    "Mung_bean_sprouts": 23,
    "Pork_belly": 24,
    "Lettuce": 25,
    "Salmon_sashimi": 26,
    "Ham": 27,
    "Mushroom": 28,
    "Lemon": 29,
    "Cucumber": 30,
    "Zucchini": 31,
    "Slice_cheese": 32,
    "Gochujang": 33,
    "Parsley": 34,
    "Grilled_beef": 35,
    "Chili_sauce": 36,
    "Avocado": 37,
    "Bread": 38,
    "Chicken": 39,
    "Baguette": 40,
    "Bell_pepper": 41,
    "Whipping_cream": 42,
    "Sundae_(Korean_sausage)": 43,
    "Chicken_breast": 44,
    "Pepper": 45,
    "Green_bell_pepper": 46,
    "Enoki_mushrooms": 47,
    "Banana": 48,
    "Almond": 49,
    "Tomato_pasta_sauce": 50,
    "Sugar": 51,
    "Canned_tuna": 52,
    "Chili_pepper": 53,
    "Somen": 54,
    "Carrot": 55,
    "Green_chili": 56,
    "Tomato": 57,
    "Celery": 58,
    "Radish_kimchi": 59,
    "Sesame_seeds": 60,
    "Napa_cabbage": 61,
    "Honey": 62,
    "Mayonnaise": 63,
    "Egg": 64,
    "Kelp": 65,
    "Ketchup": 66,
    "Jujube": 67,
    "Salt": 68,
    "Cabbage_kimchi": 69,
    "Garlic": 70,
    "Oyster_mushrooms": 71,
    "Cabbage": 72,
    "Crab_sticks": 73,
    "Hot_sauce": 74,
    "Spaghetti": 75,
    "Broccoli": 76,
    "Cherry_tomatoes": 77,
    "Fish": 78,
    "Potato": 79,
    "Cheese": 80,
    "Bacon": 81,
    "Red_cabbage": 82,
    "rice": 83,
    "Green_onion": 84,
    "Seasoned_seaweed": 85,
    "Steak": 86,
    "Seaweed": 87,
    "Green_olives": 88,
    "Onion": 89,
    "Mackerel": 90,
    "Eggplant": 91,
    "Udon": 92,
    "Butter": 93,
    "Red_bell_pepper": 94
}

# Cypher 쿼리 생성
cypher_query = "CREATE " + ", ".join(
    [f'(i{index}:Ingredient {{name: "{name}"}})' for name, index in ingredients.items()]
) + ";"

# Cypher 쿼리 출력
print(cypher_query)


CREATE (i0:Ingredient {name: "Romaine_lettuce"}), (i1:Ingredient {name: "Spinach"}), (i2:Ingredient {name: "Wakame"}), (i3:Ingredient {name: "Chili_powder"}), (i4:Ingredient {name: "Yellow_bell_pepper"}), (i5:Ingredient {name: "Cooked_rice"}), (i6:Ingredient {name: "Paprika"}), (i7:Ingredient {name: "Vienna_sausage"}), (i8:Ingredient {name: "Sausage"}), (i9:Ingredient {name: "Tofu"}), (i10:Ingredient {name: "Salsa"}), (i11:Ingredient {name: "Noodle"}), (i12:Ingredient {name: "Ramen"}), (i13:Ingredient {name: "Soybean_sprouts"}), (i14:Ingredient {name: "Jalapeño"}), (i15:Ingredient {name: "Milk"}), (i16:Ingredient {name: "Soy_sauce"}), (i17:Ingredient {name: "Shiitake_mushrooms"}), (i18:Ingredient {name: "Basil"}), (i19:Ingredient {name: "Sweet_potato"}), (i20:Ingredient {name: "Squid"}), (i21:Ingredient {name: "Shrimp"}), (i22:Ingredient {name: "Rice"}), (i23:Ingredient {name: "Mung_bean_sprouts"}), (i24:Ingredient {name: "Pork_belly"}), (i25:Ingredient {name: "Lettuce"}), (i26:Ingredi